## This notebook prepares the data for applying the Regression algorithms
### and employs the following Constructs..
### groupby(), agg(), sort_values(), reset_index()
### pd.get_dummies : Convert a categorical variable into dummy/indicator variables (like OneHotEncoder)

In [1]:
#######Walmart sales - data pre-processing START

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from IPython.display import display

In [2]:
#Let's first build a single dataframe with train and prediction dates so that any feature processing can be performed consistently

df_train = pd.read_csv('Walmart/train.csv')
# df_train = pd.read_excel('Walmart/train.xlsx')
df_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
Store           421570 non-null int64
Dept            421570 non-null int64
Date            421570 non-null object
Weekly_Sales    421570 non-null float64
IsHoliday       421570 non-null bool
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


In [3]:
# aggregate data per store and flag rows to differentiate from prediction placeholders

df_train = df_train.rename(columns={'Weekly_Sales': 'WeeklySales'})

df_train = df_train.groupby(['Store','Date','IsHoliday'])['WeeklySales'].agg(np.mean).to_frame().reset_index()  #its mean based on mentioned columns and the mean value is updated to WeeklySales columns; removing Dept column as well.

df_train.WeeklySales = df_train.WeeklySales.map(int)  #just to remove decimals.

df_train.head()

,Store,Date,IsHoliday,WeeklySales
0,1,1/13/2012,False,20557
1,1,1/14/2011,False,19591
2,1,1/20/2012,False,19101
3,1,1/21/2011,False,18436
4,1,1/27/2012,False,18847


In [4]:
df_train['Train'] = True

# test dataset
df_pred = pd.read_csv('Walmart/test.csv').drop(['Dept'], axis=1).drop_duplicates()
# df_pred = pd.read_excel('Walmart/test.xlsx').drop(['Dept'], axis=1).drop_duplicates()
df_pred['Train'] = False

df = pd.concat([df_train, df_pred])
df.sort_values(by=['Date','Store'], inplace=True)
df.reset_index(drop=True, inplace=True)

print("train: %s %s" % (df_train.Date.min(), df_train.Date.max()))
print("pred: %s %s" % (df_pred.Date.min(), df_pred.Date.max()))

train: 1/13/2012 9/9/2011
pred: 1/11/2013 7/5/2013


In [5]:
df_train.head()

,Store,Date,IsHoliday,WeeklySales,Train
0,1,1/13/2012,False,20557,True
1,1,1/14/2011,False,19591,True
2,1,1/20/2012,False,19101,True
3,1,1/21/2011,False,18436,True
4,1,1/27/2012,False,18847,True


In [6]:
df_pred.head()

,Store,Date,IsHoliday,Train
0,1,11/2/2012,False,False
1,1,11/9/2012,False,False
2,1,11/16/2012,False,False
3,1,11/23/2012,True,False
4,1,11/30/2012,False,False


In [7]:
del(df_train)
del(df_pred)

In [8]:
#Break dates into week, year and Year-week

df['DateString'] = df['Date']
df['Date'] = pd.to_datetime(df['Date'])

df['Week'] = df['Date'].dt.week
df['Year'] = df['Date'].dt.year
df['YearWeek'] = df.Year.map(str) + "-" + df.Week.map(lambda v: "%02d"%v)
df.head(5)

,Date,IsHoliday,Store,Train,WeeklySales,DateString,Week,Year,YearWeek
0,2013-01-11,False,1,False,NaN,1/11/2013,2,2013,2013-02
1,2013-01-11,False,2,False,NaN,1/11/2013,2,2013,2013-02
2,2013-01-11,False,3,False,NaN,1/11/2013,2,2013,2013-02
3,2013-01-11,False,4,False,NaN,1/11/2013,2,2013,2013-02
4,2013-01-11,False,5,False,NaN,1/11/2013,2,2013,2013-02


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 9 columns):
Date           8190 non-null datetime64[ns]
IsHoliday      8190 non-null bool
Store          8190 non-null int64
Train          8190 non-null bool
WeeklySales    6435 non-null float64
DateString     8190 non-null object
Week           8190 non-null int64
Year           8190 non-null int64
YearWeek       8190 non-null object
dtypes: bool(2), datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 464.0+ KB


In [11]:
#Creating a new column with the sales increase from previous year ; playing only with Store and WeeklySales
df['WeeklySalesPreviousYear'] = np.nan

In [12]:
for store in df.Store.unique():
    idx = df.Store==store
    a = df['WeeklySales'][idx].values
    a = np.roll(a,52) # shift by one year
    a[0:52] = np.nan
    df.loc[idx, 'WeeklySalesPreviousYear'] = a

In [13]:
a

array([   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan, 10098.,  9909.,    nan,
       10150.,  9761.,    nan,  9314.,  9351.,    nan, 10501., 10306.,
       10298., 11128., 10602., 10675., 10880., 11348., 11074., 11347.,
       11667., 10621., 10786., 11080., 10967., 11891., 11427.,    nan,
       11376., 10805.,    nan,    nan, 17215., 17389.,    nan, 12256.,
       11403.,    nan, 14960.,    nan, 15866., 16765., 12509.,    nan,
       21742., 24389.,    nan, 12930., 12785., 10136.,    nan, 14075.,
          nan, 11818., 11108.,  9661.,    nan, 12630., 11973., 12192.,
      

In [15]:
df

,Date,IsHoliday,Store,Train,WeeklySales,DateString,Week,Year,YearWeek,WeeklySalesPreviousYear
0,2013-01-11,False,1,False,NaN,1/11/2013,2,2013,2013-02,NaN
1,2013-01-11,False,2,False,NaN,1/11/2013,2,2013,2013-02,NaN
2,2013-01-11,False,3,False,NaN,1/11/2013,2,2013,2013-02,NaN
3,2013-01-11,False,4,False,NaN,1/11/2013,2,2013,2013-02,NaN
4,2013-01-11,False,5,False,NaN,1/11/2013,2,2013,2013-02,NaN
5,2013-01-11,False,6,False,NaN,1/11/2013,2,2013,2013-02,NaN
6,2013-01-11,False,7,False,NaN,1/11/2013,2,2013,2013-02,NaN
7,2013-01-11,False,8,False,NaN,1/11/2013,2,2013,2013-02,NaN
8,2013-01-11,False,9,False,NaN,1/11/2013,2,2013,2013-02,NaN
9,2013-01-11,False,10,False,NaN,1/11/2013,2,2013,2013-02,NaN


In [16]:
df['WeeklySalesIncreaseRatio'] = df['WeeklySales'] / df['WeeklySalesPreviousYear']

In [17]:
df.tail()

,Date,IsHoliday,Store,Train,WeeklySales,DateString,Week,Year,YearWeek,WeeklySalesPreviousYear,WeeklySalesIncreaseRatio
8185,2011-09-09,True,41,True,17791.0,9/9/2011,36,2011,2011-36,19255.0,0.923968
8186,2011-09-09,True,42,True,12674.0,9/9/2011,36,2011,2011-36,10385.0,1.220414
8187,2011-09-09,True,43,True,14425.0,9/9/2011,36,2011,2011-36,12776.0,1.129070
8188,2011-09-09,True,44,True,5800.0,9/9/2011,36,2011,2011-36,6344.0,0.914250
8189,2011-09-09,True,45,True,11136.0,9/9/2011,36,2011,2011-36,12096.0,0.920635


In [19]:
(143+39) * 45

8190

In [20]:
#Creating dummy variables for Store, and mapping IsHoliday to integers
stores = df.Store
df = pd.get_dummies(df, columns=['Store'])
df['Store'] = stores
df['IsHolidayInt'] = df.IsHoliday.map(int)

df.sample(5)

,Date,IsHoliday,Train,WeeklySales,DateString,Week,Year,YearWeek,WeeklySalesPreviousYear,WeeklySalesIncreaseRatio,...,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45,Store,IsHolidayInt
5654,2012-06-15,False,True,9135.0,6/15/2012,24,2012,2012-24,8509.0,1.073569,...,0,0,0,0,0,0,0,0,30,0
4001,2012-04-13,False,True,12531.0,4/13/2012,15,2012,2012-15,12291.0,1.019526,...,0,0,0,0,1,0,0,0,42,0
2134,2010-12-03,False,True,33096.0,12/3/2010,48,2010,2010-48,NaN,NaN,...,0,0,0,0,0,0,0,0,20,0
4396,2012-04-27,False,True,15872.0,4/27/2012,17,2012,2012-17,27071.0,0.586310,...,0,0,0,0,0,0,0,0,32,0
3834,2012-03-09,False,True,27392.0,3/9/2012,10,2012,2012-10,40832.0,0.670846,...,0,0,0,0,0,0,0,0,10,0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 57 columns):
Date                        8190 non-null datetime64[ns]
IsHoliday                   8190 non-null bool
Train                       8190 non-null bool
WeeklySales                 6435 non-null float64
DateString                  8190 non-null object
Week                        8190 non-null int64
Year                        8190 non-null int64
YearWeek                    8190 non-null object
WeeklySalesPreviousYear     4365 non-null float64
WeeklySalesIncreaseRatio    3735 non-null float64
Store_1                     8190 non-null uint8
Store_2                     8190 non-null uint8
Store_3                     8190 non-null uint8
Store_4                     8190 non-null uint8
Store_5                     8190 non-null uint8
Store_6                     8190 non-null uint8
Store_7                     8190 non-null uint8
Store_8                     8190 non-null uint8
Store_9       

In [23]:
df.to_csv('Walmart/preprocessed.csv')   
#This files contains both Train and test dataset with average weekly sales.
#######Walmart sales - data pre-processing END